In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import sys
import os

# Add the project root to sys.path to allow imports from src
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

from src.model.algorithms.xgboost_model import XGBoostModel

# Get data from cache
df = pd.read_csv(os.path.join(project_root, "data", "cache", "NVDA.csv"))

df = df.iloc[-500:]

df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

X, y = df, df['close'].shift(-1)

X, y



(           ticker    open     high     low   close  adjClose     volume  \
 date                                                                      
 2023-12-07   NVDA   45.70   46.630   45.60   46.60     46.57  350823000   
 2023-12-08   NVDA   46.60   47.740   46.55   47.51     47.48  359224000   
 2023-12-11   NVDA   47.49   47.530   45.83   46.63     46.60  509728090   
 2023-12-12   NVDA   46.05   47.670   46.05   47.66     47.63  372387000   
 2023-12-13   NVDA   47.63   48.590   47.61   48.09     48.06  447792000   
 ...           ...     ...      ...     ...     ...       ...        ...   
 2025-11-28   NVDA  179.01  179.290  176.50  177.00    177.00  121332800   
 2025-12-01   NVDA  174.76  180.300  173.68  179.92    179.92  188131000   
 2025-12-02   NVDA  181.76  185.660  180.00  181.46    181.46  182632200   
 2025-12-03   NVDA  181.08  182.450  179.11  179.59    179.59  164721400   
 2025-12-04   NVDA  181.57  184.515  179.97  183.46    183.46  162435495   
 
          

In [2]:
# --- 2. Initialize Model & Test _prepare_data ---
model = XGBoostModel(n_estimators=10, max_depth=3)

# Call internal _prepare_data directly to inspect DMatrix
dtrain = model._prepare_data(X, y)

type(dtrain)


xgboost.core.DMatrix

In [3]:

# --- 3. Verify Feature Transformations ---
expected_feats = ['log_ret', 'vol_rel', 'dist_vwap', 'day_sin', 'log_ret_lag_1']
missing = [f for f in expected_feats if f not in dtrain.feature_names]

if not missing:
    print("\n✅ All expected features present.")
else:
    print("\n❌ Missing features:", missing)

# --- 4. Test Training Loop ---
print("\n--- Training ---")
try:
    model.train(df, y)
    print("✅ Training completed successfully.")
except Exception as e:
    print(f"❌ Training failed: {e}")

# --- 5. Test Prediction ---
print("\n--- Prediction ---")
try:
    # Predict on the last 5 rows of original df
    preds = model.predict(df.tail(5))
    print("Predictions (Log Returns):", preds)
except Exception as e:
    print(f"❌ Prediction failed: {e}")


✅ All expected features present.

--- Training ---
✅ Training completed successfully.

--- Prediction ---
Predictions (Log Returns): []


/Users/brader/Desktop/stock-dashboard/venv/lib/python3.13/site-packages/xgboost/core.py:774: UserWarning: [17:12:54] WARNING: /Users/runner/work/xgboost/xgboost/src/common/error_msg.cc:56: Empty dataset at worker: 0
  return func(**kwargs)
